## BERT PIPELINE

In [4]:
# Imports
!pip install matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

     ---------------------------------------- 7.2/7.2 MB 5.8 MB/s eta 0:00:00
     -------------------------------------- 164.1/164.1 kB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 55.3/55.3 kB 2.8 MB/s eta 0:00:00
     -------------------------------------- 960.8/960.8 kB 7.6 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 6.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: C:\Users\Adrian Bajraktari\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [10]:
# Load the dataset from google drive
!pip install google.colab
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_excel('/content/drive/MyDrive/KANO Modell Studie/Datasets/LabeledDatasets/Dataset_final.xlsx')

df = df[df['final_Label'].notna()]
df = df[df['review'].notna()]
df = df.drop(columns = ['Unnamed: 0'])
df.info()

  Using cached google-colab-1.0.0.tar.gz (72 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): still running...
  Preparing metadata (setup.py): finished with status 'done'
  Using cached jupyter_client-7.4.3-py3-none-any.whl (132 kB)
  Using cached jupyter_client-7.4.2-py3-none-any.whl (132 kB)
  Using cached jupyter_client-7.4.1-py3-none-any.whl (132 kB)
  Using cached jupyter_client-7.4.0-py3-none-any.whl (132 kB)
  Using cached jupyter_client-7.3.4-py3-none-any.whl (132 kB)
  Using cached jupyter_client-7.3.3-py3-none-any.whl (131 kB)
  Using cached jupyter_client-7.3.2-py3-none-any.whl (131 kB)
  Using cached jupyter_client-7.3.1-py3-none-

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [1130 lines of output]
  C:\Users\Adrian Bajraktari\AppData\Local\Temp\pip-install-qo0dwkft\pandas_12a1de88a26f48a3a973861f95408ac1\setup.py:50: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
    _CYTHON_INSTALLED = ver >= LooseVersion(min_cython_ver)
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\pandas
  copying pandas\conftest.py -> build\lib.win-amd64-cpython-310\pandas
  copying pandas\testing.py -> build\lib.win-amd64-cpython-310\pandas
  copying pandas\_version.py -> build\lib.win-amd64-cpython-310\pandas
  copying pandas\__init__.py -> build\lib.win-amd64-cpython-310\pandas
  creating build\lib.win-amd64-cpython-310\pandas\api
  copying pandas\api\__init__.py -> build\lib.win-amd64-cpython-310\pandas\api
  

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Visualize the distribution of reviews

plt.style.use('ggplot')

num_classes = len(df['final_Label'].value_counts())

colors = plt.cm.Dark2(np.linspace(0, 1, num_classes))
iter_color = iter(colors)

df['final_Label'].value_counts().plot.barh(title='Reviews for each label (n, %)', 
                                                 ylabel='Label after Kano',
                                                 color=colors,
                                                 figsize=(9,9))

for i, v in enumerate(df['final_Label'].value_counts()):
  c = next(iter_color)
  plt.text(v, i,
           " "+str(v)+", "+str(round(v*100/df.shape[0],2))+"%", 
           color=c, 
           va='center', 
           fontweight='bold')

In [ ]:
# Encode the labels

possible_labels = df.final_Label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
df['label'] = df.final_Label.replace(label_dict)
label_dict

In [ ]:
df.head()

In [ ]:
# Downsampling 

# Save different labels in seperate dataframes
df_B = df[df['label'] == 0] # B
df_P = df[df['label'] == 3] # P
df_D = df[df['label'] == 1] # D
df_I = df[df['label'] == 2] # I

# Reduce the number of samples to the number of the class with the least samples
df_I_downsampled = df_I.sample(df_D.shape[0]) 
df_B_downsampled = df_B.sample(df_D.shape[0]) 
df_P_downsampled = df_P.sample(df_D.shape[0]) 

# Combine the dataframes
df_downsampled = pd.concat([df_I_downsampled, df_B_downsampled, df_P_downsampled, df_D])
df=df_downsampled

df=shuffle(df)
df

In [ ]:
print(df['label'].value_counts())

In [ ]:
# Imports
!pip install transformers
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

In [ ]:
# Initialize Tokenizer to prepare the inputs for the model

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
# Tokenize the reviews

def generate_training_data(df, ids, masks, tokenizer):
  for i, text in tqdm(enumerate(df['review'])):
    tokenized_text = tokenizer.encode_plus(
        text, 
        max_length = 256,
        truncation = True,
        padding = 'max_length',
        add_special_tokens = True,
        return_tensors = 'tf'
    )
    ids[i, :] = tokenized_text.input_ids
    masks[i, :] = tokenized_text.attention_mask
  return ids, masks

In [ ]:
# Change how the dataset is returned

def MapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

# BERT

In [ ]:
# Create BERT model

from transformers import TFBertModel

model =  TFBertModel.from_pretrained('bert-base-cased')

In [ ]:
def prepareDataframe(df):
  # Create input IDs and attention masks
  X_input_ids = np.zeros((len(df), 256))
  X_attn_masks = np.zeros((len(df), 256))
  X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)
  # Create a one hot encoded target vector of length 4 for each review (4 classes)
  labels = np.zeros((len(df), 4))
  # For each review, the one hot encoded vector contains the information about the label it has 
  labels[np.arange(len(df)), df['label'].values] = 1
  dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
  dataset = dataset.map(MapFunction)
  dataset = dataset.shuffle(10000).batch(16, drop_remainder = True)
 
  return dataset

In [ ]:
def setUpBERT():
  # todo: try 512
  input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
  attention_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')
  bert_embds = model.bert(input_ids, attention_mask=attention_masks)[1]
  # todo: try multiple intermediate layers
  intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
  # todo: try softmax
  output_layer = tf.keras.layers.Dense(4, activation='sigmoid', name='output_layer')(intermediate_layer)
  classification_model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=output_layer)

  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-2)
  loss_function = tf.keras.losses.CategoricalCrossentropy()
  accuracy = tf.keras.metrics.CategoricalAccuracy('accuracy')
  classification_model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
  # Model is compiled, now train it on the dataset
  return classification_model

In [ ]:
def prepare_data(input_text, tokenizer):
  token = tokenizer.encode_plus(
      input_text, 
      max_length = 256,
      truncation = True,
      padding = 'max_length',
      add_special_tokens = True,
      return_tensors = 'tf'
  )
  return{
      'input_ids': tf.cast(token.input_ids, tf.float64),
      'attention_mask': tf.cast(token.attention_mask, tf.float64)
  }

In [ ]:
# Returns the label with the highest probability
def prediction(model, process, classes=possible_labels):
    probability = model.predict(process)[0]
    return classes[np.argmax(probability)]

In [ ]:
### K-fold cross validation ###
import os
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.layers import Dense, Activation
from sklearn.metrics import roc_auc_score,auc,roc_curve,accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

kf = StratifiedKFold(n_splits = 10)

X = df[['review','final_Label', 'label']]
y = df.final_Label

result_kfold = []
y_train = []
y_test = []
fold = 0

for train, test in kf.split(X, df['final_Label']):
  fold += 1
  print(f'Fold #{fold}')
  
  x_train, x_test = X.iloc[train], X.iloc[test]
  
  x_train = prepareDataframe(x_train)
  x_test = prepareDataframe(x_test)
  
  myModel = setUpBERT()
  myModel.fit(x_train, validation_data=(x_test), verbose=0, epochs=1)

  for i in test:
    y_train.append(prediction(myModel,prepare_data(X.review.iloc[i], tokenizer=tokenizer),classes=possible_labels))
    y_test.append(X.final_Label.iloc[i])
    result_kfold.append(accuracy_score(y_test, y_train))

  result_kfold.append(accuracy_score(y_test, y_train))
  print(classification_report(y_test, y_train,target_names=possible_labels))

  actual_label = np.zeros((len(y_test), 4))
  actual_label_dt = pd.DataFrame(y_test).replace(label_dict)
  actual_label[np.arange(len(y_test)), actual_label_dt[0].values] = 1
  actual_label.shape

  predicted_label = np.zeros((len(y_train), 4))
  predicted_label_dt = pd.DataFrame(y_train).replace(label_dict)
  predicted_label[np.arange(len(y_train)), predicted_label_dt[0].values] = 1
  predicted_label.shape

  auc_score = roc_auc_score(actual_label, predicted_label, multi_class='ovo', average='weighted')
  print(f"ROC AUC Score  = {auc_score}")

In [ ]:
print(f'Durchschnittliche accuracy = {np.mean(result_kfold)}')

## Training on the dataset at hand, testing on the 'Trainingskorpus' dataset

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
testset = pd.read_excel('/content/drive/MyDrive/KANO Modell Studie/Datasets/LabeledDatasets/Trainingskorpus_Final.xlsx')
testset.head()

In [ ]:
# Test on the Trainingskorpus dataset

agreed1 = 0

for txt, label in zip(testset.data, testset.final_Label):
    
    process = prepare_data(txt, tokenizer)
    result = prediction(myModel, process=process)

    if(result == label):
        agreed1 += 1
        
print(str(agreed1) + ' / ' + str(testset.shape[0]))

In [ ]:
# Accuracy
print('Accuracy Trainingskorpus: ' + str(agreed1/testset.shape[0]*100) + '%')

In [ ]:
### Train test split without k-fold cross validation ###

# p = 0.8
# training_size = int((len(df)//16)*p)

# Split dataset into training and testing sets

# train_dataset = dataset.take(training_size)
# test_dataset = dataset.skip(training_size)

# hist = classification_model.fit(
#     train_dataset,
#     validation_data = test_dataset,
    # todo: try other values 
#     epochs = 20
# )

# Save the model
# classification_model.save('KANO_classification_model')

# Load the model
# loaded_model = tf.keras.models.load_model('KANO_classification_model')


# Test model on input text
# create input text
# input_text = 'Worst app ever'

# def prepare_data(input_text, tokenizer):
#  token = tokenizer.encode_plus(
#      input_text, 
#      max_length = 256,
#      truncation = True,
#      padding = 'max_length',
#      add_special_tokens = True,
#      return_tensors = 'tf'
#  )
#  return{
#      'input_ids': tf.cast(token.input_ids, tf.float64),
#      'attention_mask': tf.cast(token.attention_mask, tf.float64)
#  }

# tokenized_input_text = prepare_data(input_text, tokenizer)

# probs = loaded_model.predict(tokenized_input_text)

# classes = ['Basic', 'Delighter', 'Irrelevant', 'Performance']

# Get the maximum value of probabilities for the input text to be in certain class
# output_index = np.argmax(probs[0])
# print(classes[output_index])

# {'B': 0, 'D': 1, 'I': 2, 'P': 3}


## Combine dataset at hand and 'Trainingskorpus' dataset, then do Train-Test-Split